In [ ]:
import pandas as pd
import numpy as np


file='C:/Users/VIP13/Downloads/Все по нейронкам/all_rv.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

# Print the sheet names
print(xl.sheet_names)
df1 = xl.parse(xl.sheet_names[0])



In [ ]:
df1.head()

In [ ]:
# вгрузил данные как надо, разбил на тест и трейн, отнормировал
from sklearn import preprocessing
# uni_data = df1['ln_rv_btc'].copy()

uni_data = np.log(df1['rv_shk_x'].copy())
uni_data.index = df1['Unnamed: 0']
uni_data.head()
xData=np.array(uni_data)
xData = xData.reshape(len(xData),1)
dataset=xData

scaler = preprocessing.MinMaxScaler() 

dataset = scaler.fit_transform(dataset)

#разбил на трен и тест
coef_train_test=0.7

train_size = int(len(dataset) * coef_train_test)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# разбил трен на трен и вал
# coef_train_val=0.9

# train_size = int(len(train) * coef_train_val)
# val_size = len(train) - train_size

# train1, val= train[0:val_size,:], train[train_size:len(dataset),:]

In [ ]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
#здесь sequence - это выборка, которую разбиваем, n_steps - то самое "скользящее окно"
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# # define input sequence
# raw_seq = train
# # choose a number of time steps
# n_steps = 250
# # split into samples
# X, y = split_sequence(raw_seq, n_steps)
# # summarize the data
# # for i in range(len(X)):
# # 	print(X[i], y[i])

In [ ]:
n_steps = 100#хорошо работает при 250

x_train, y_train=split_sequence(train, n_steps )
x_test, y_test=split_sequence(test, n_steps)
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)
print(x_test.shape)

x_train_all=x_train.copy()
y_train_all=y_train.copy()

In [ ]:
# разбил трен на трен и вал
coef_train_val=0.9
len_train=int(len(x_train)*coef_train_val)
x_train1=x_train[0:len_train].copy()
x_val=x_train[len_train:]
x_train=x_train1

y_train1=y_train[0:int(len(y_train)*coef_train_val)].copy()
y_val=y_train[int(len(y_train)*coef_train_val):]
y_train=y_train1

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
from keras import backend as K

# def me(y_true, y_pred):
def myloss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [ ]:
# !pip install tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten, Convolution1D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import BatchNormalization


In [ ]:
# ТУТ заметки кода, можно вставлять кусками

# СНН
# model_m.add(Conv1D(5, 1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
# modelC.add(MaxPooling1D(pool_size=1))

# просто слой с числом нейронок и активатором
# model_m.add(Dense(50, activation='linear'))
# дропаут, для борьбы с переобучением
# model.add(Dropout(0.25))
# батчевая нормализация
# model_m.add(BatchNormalization())
# полносвязный слой, его лучше в конце вставлять
# model_m.add(Flatten())
# слой с регуляризацией
# model_m.add(Dense(64, input_dim=64, bias_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
# раняя остановка, чтобы не переобучаться, в мониторе писать пример 
# early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)


сравнение пошло

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from pandas import ExcelWriter
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dense, Dropout, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Flatten, Bidirectional
from keras.layers import Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.layers import Flatten, Convolution1D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model

from keras import regularizers
import time

start_time = time.time()

activation_functions=['tanh', 'linear', 'relu', 'softmax', 'elu', 'selu', 'softplus', 'softsign', 'sigmoid', 'hard_sigmoid', 'exponential', 'LeakyReLU', 'PReLU', 'ThresholdedReLU']
# activation_functions_n=['selu', 'softplus', 'softsign', 'sigmoid', 'hard_sigmoid', 'exponential', 'LeakyReLU', 'PReLU', 'ThresholdedReLU']

# loss_f=['mae', 'mse']
# T_F=['True', 'False']

# all_logir=pd.DataFrame({ })
i=0
all_logir_pred=pd.DataFrame()
all_logir_mae=pd.DataFrame()
all_logir_mse=pd.DataFrame()
all_logir_me=pd.DataFrame()

testY = scaler.inverse_transform(y_test)
testY=testY.reshape(testY.shape[0])
all_logir_pred['testY']=testY
for a_1 in activation_functions:
    for d_1 in np.arange(0, 0.3, 0.1):
        for a_2 in activation_functions:
            for d_2 in np.arange(0, 0.3, 0.1):
                for a_3 in activation_functions:
                                            
                    i=i+1
                    name_col=(str(a_1)+'-'+str(d_1)+'-'+
                            str(a_2)+'-'+str(d_2)+'-'+str(a_3)+'-'+str(i))
                            
#                     all_logir[name_col]=activation_functions
                                            
                    model_m = Sequential()                        
                    model_m.add(Dense(1, input_shape=(x_train.shape[1], x_train.shape[2])))#входной нейрон

                    model_m.add(LSTM(64,return_sequences = True, activation=a_1, input_shape = (x_train.shape[1], x_train.shape[2])))
                    model_m.add(Dropout(d_1))
                    model_m.add(LSTM(32, activation=a_2, input_shape = (x_train.shape[1], x_train.shape[2])))
                    model_m.add(Dropout(d_2))

                    model_m.add(Dense(10, activation=a_3))
                    model_m.add(Flatten())
                    model_m.add(Dense(1, activation='linear'))

                    model_m.compile(optimizer='adam', loss='mse')

                    early_stop = EarlyStopping(monitor = 'loss', patience = 5)

                    history = model_m.fit(x_train, 
                                                            y_train, 
                                                            epochs=20, 
                                                            batch_size=15, #для борьбы с затухающим градиентом, как часто перетасовывать говну данных
                                                            verbose=1,
                                                            shuffle = False,
                                                            callbacks = [early_stop],
                                                            validation_data=(x_val, y_val)
                                                            )
                    predicted = model_m.predict(x_test)
                    testPredict = scaler.inverse_transform(predicted)
                    testPredict=testPredict.reshape(testPredict.shape[0])
                    all_logir_pred[str(name_col)]=testPredict

# predicted.reshape(predicted.shape[0])
                    me=testY-testPredict
                    mae=abs(testY-testPredict)
                    mse=(testY-testPredict)**2
                            
                
#                     testY.reshape(testY.shape[0])
                    all_logir_me[str(name_col)]=me.reshape(me.shape[0])
                    all_logir_mae[str(name_col)]=mae.reshape(mae.shape[0])
                    all_logir_mse[str(name_col)]=mse.reshape(mse.shape[0])
                    
                    if i % 200 == 0:
                        writer_pred=pd.ExcelWriter('all_logir_pred.xlsx')
                        all_logir_pred.to_excel(writer_pred)
                        writer_pred.save()
#                         writer_pred.close()
                        
                        writer_me=pd.ExcelWriter('all_logir_me.xlsx')
                        all_logir_me.to_excel(writer_me)
                        writer_me.save()
#                         writer_me.close()
                        
                        writer_mae=pd.ExcelWriter('all_logir_mae.xlsx')
                        all_logir_mae.to_excel(writer_mae)
                        writer_mae.save()
#                         writer_mae.close()
                        
                        writer_mse=pd.ExcelWriter('all_logir_mse.xlsx')
                        all_logir_mse.to_excel(writer_mse)
                        writer_mse.save()
#                         writer_mse.close()
                        
                    print(i)
                





In [ ]:
from pandas import ExcelWriter

print(i)
writer_pred=pd.ExcelWriter('all_logir_pred.xlsx')
all_logir_pred.to_excel(writer_pred)
writer_pred.save()
writer_pred.close()
                        
writer_me=pd.ExcelWriter('all_logir_me.xlsx')
all_logir_me.to_excel(writer_me)
writer_me.save()
writer_me.close()
                        
writer_mae=pd.ExcelWriter('all_logir_mae.xlsx')
all_logir_mae.to_excel(writer_mae)
writer_mae.save()
writer_mae.close()
                        
writer_mse=pd.ExcelWriter('all_logir_mse.xlsx')
all_logir_mse.to_excel(writer_mse)
writer_mse.save()
writer_mse.close()

end_time=time.time() - start_time    
print(end_time)
print(i)
print(all_logir_pred)